In [1]:
import os, sys
import numpy as np
import pandas as pd
import torch
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
sys.path.insert(0, 'src')
from utils.places365_pred_utils import get_class_category_dict, get_category_class_dict
from utils.utils import ensure_dir, write_lists

In [2]:
N_ATTR = 1200  # Taken from Vikram


### Load attributes and predictions

In [3]:
data_path = os.path.join('data', 'ade20k', 'ade20k_imagelabels.pth')
save_dir = os.path.join('saved', 'ADE20K', '0501_105640')
predictions_path = os.path.join(save_dir, '{}_logits_predictions.pth')

data = torch.load(data_path)
print("Data keys: {}".format(data.keys()))
train = torch.load(predictions_path.format('train'))
train_paths = train['paths']
train_predictions = train['predictions']

val = torch.load(predictions_path.format('val'))
val_paths = val['paths']
val_logits = val['logits']
val_predictions = val['predictions']

test = torch.load(predictions_path.format('test'))
test_paths = test['paths']
test_predictions = test['predictions']

print(train_predictions.shape, val_predictions.shape, test_predictions.shape)
predictions = {
    'train': train_predictions,
    'val': val_predictions,
    'test': test_predictions
}
paths = {
    'train': train_paths,
    'val': val_paths,
    'test': test_paths
}

Data keys: dict_keys(['train', 'val', 'test', 'labels'])
(13326,) (4442,) (4442,)


In [4]:
def get_one_hot_attributes(data, paths):
    splits = ['train', 'val', 'test']
    attributes = {
        'train': [],
        'val': [], 
        'test': []
    }
    
    for split in splits:
        split_paths = paths[split]
        print("Processing attributes for {} split".format(split))
        for path in tqdm(split_paths):
            # Obtain attributes and covnvert to one hot
            cur_attributes = data['labels'][path]
            one_hot_attributes = np.zeros(N_ATTR)
            one_hot_attributes[cur_attributes] = 1
            attributes[split].append(one_hot_attributes)
        attributes[split] = np.stack(attributes[split], axis=0)
        
    return attributes


attributes = get_one_hot_attributes(data, paths)

Processing attributes for train split


100%|██████████████████████████████████| 13326/13326 [00:00<00:00, 92202.73it/s]

Processing attributes for val split



100%|███████████████████████████████████| 4442/4442 [00:00<00:00, 142337.30it/s]


Processing attributes for test split


100%|███████████████████████████████████| 4442/4442 [00:00<00:00, 130098.17it/s]


In [6]:
train_attributes = attributes['train']
counts = np.sum(train_attributes, axis=0)
print("{} concepts that occur > 150 times in training".format(len(np.where(counts > 150)[0])))
print("{} concepts that occur at all in training".format(len(np.nonzero(counts)[0])))

182 concepts that occur > 150 times in training
623 concepts that occur at all in training


### Obtain attributes that are frequently occuring in the training set

In [5]:
FREQUENCY_THRESH = 150
n_samples = attributes['train'].shape[0]
train_counts = np.sum(attributes['train'], axis=0)

def obtain_frequent_attributes(cur_attributes, train_counts):

    # Obtain one-hot encoding of attributes that exceed frequency threshold
    freq_attributes_one_hot = np.where(train_counts > FREQUENCY_THRESH, 1, 0)
    # Mask out infrequent attributes
    freq_attributes = np.where(freq_attributes_one_hot == 1, cur_attributes, 0)
    # freq_train_attributes = np.where(freq_attributes_one_hot == 1, attributes['train'], 0)
    # freq_val_attributes = np.where(freq_attributes_one_hot == 1, attributes['val'], 0)
    # freq_test_attributes = np.where(freq_attributes_one_hot == 1, attributes['test'], 0)

    # Sanity checks
    discarded_attributes_idxs = np.nonzero(np.where(train_counts < FREQUENCY_THRESH, 1, 0))[0]
    kept_attributes_idxs = np.nonzero(train_counts > FREQUENCY_THRESH)[0]
    assert (kept_attributes_idxs == np.nonzero(freq_attributes_one_hot)[0]).all()

    zeroed_ctr = 0
    ctr = 0
    for idx, (orig, new) in enumerate(zip(cur_attributes, freq_attributes)):
        # print(orig
        if not (orig == new).all():
            orig_idxs = np.nonzero(orig)[0]
            new_idxs = np.nonzero(new)[0]
            # Assert new idxs ONLY contains the kept attributes and none of discarded
            assert len(np.intersect1d(new_idxs, discarded_attributes_idxs)) == 0
            assert len(np.intersect1d(new_idxs, kept_attributes_idxs)) == len(new_idxs)
            # Assert overlap with original indices is equal to new indices
            assert (np.intersect1d(orig_idxs, new_idxs) == new_idxs).all()
            if len(new_idxs) == 0:
                zeroed_ctr += 1
            ctr += 1
    print("{} examples have no more attributes".format(zeroed_ctr))
    print("{}/{} examples affected".format(ctr, len(cur_attributes)))
    
    return freq_attributes

freq_attributes = {}
for split in ['train', 'val', 'test']:
    cur_freq_attributes = obtain_frequent_attributes(
        cur_attributes=attributes[split],
        train_counts = train_counts)
    freq_attributes[split] = cur_freq_attributes


6 examples have no more attributes
8308/13326 examples affected
3 examples have no more attributes
2736/4442 examples affected
3 examples have no more attributes
2804/4442 examples affected


#### Obtain scene category predictions and save to save_dir

In [6]:
scene_category_dict = get_class_category_dict()

def get_scene_category_predictions(scene_predictions, scene_category_dict):
    category_predictions = []
    for scene_prediction in scene_predictions:
        category_predictions.append(scene_category_dict[scene_prediction])
    
    category_predictions = np.array(category_predictions)
    return category_predictions

# Save category predictions in save_dir
category_predictions = {}
for split, split_predictions in predictions.items():
    split_category_predictions = get_scene_category_predictions(
        scene_predictions=split_predictions,
        scene_category_dict=scene_category_dict)
    
    save_path = os.path.join(save_dir, '{}_scene_category_predictions.pth'.format(split))
    if os.path.exists(save_path):
        print("{} already exists.".format(save_path))
    else:
        torch.save({'scene_category_predictions': split_category_predictions}, save_path)
        print("Saved scene category predictions for {} to {}".format(split, save_path))
    category_predictions[split] = split_category_predictions
    

saved/ADE20K/0501_105640/train_scene_category_predictions.pth already exists.
saved/ADE20K/0501_105640/val_scene_category_predictions.pth already exists.
saved/ADE20K/0501_105640/test_scene_category_predictions.pth already exists.


### Run hyperparameter search on logistic regression for linear model to predict classes from attributes

In [100]:
def hyperparam_search_l1(train_features, train_labels, val_features, val_labels, 
                      Cs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 3, 5]):
    best_clf = None
    best_acc = 0
    
    for c in Cs:
        clf = LogisticRegression(solver='liblinear', C=c, penalty='l1')
        clf.fit(train_features, train_labels)
        score = clf.score(val_features, val_labels)
        if score>best_acc:
            best_acc = score
            best_clf = clf
            print("Best accuracy: {} Regularization: {}".format(score, c))
    
    return best_clf


#### Predicting all 365 classes with all part/object attributes

In [33]:
hyperparam_search_l1(
    train_features=attributes['train'],
    train_labels=train_predictions,
    val_features=attributes['val'],
    val_labels=val_predictions)
    

0.029941467807294012 0.001
0.1154885186852769 0.005
0.15038271049076993 0.01
0.23660513282305268 0.05
0.28140477262494373 0.1
0.3685276902296263 0.5
0.375056280954525 1


LogisticRegression(C=1, penalty='l1', solver='liblinear')

#### Predicting all  365 classes with part/object attributes that appear in >=150 training examples

In [77]:
hyperparam_search_l1(
    train_features=freq_train_attributes,
    train_labels=train_predictions,
    val_features=freq_val_attributes,
    val_labels=val_predictions)

Best accuracy: 0.029941467807294012 Regularization: 0.001
Best accuracy: 0.1154885186852769 Regularization: 0.005
Best accuracy: 0.14678072940117065 Regularization: 0.01
Best accuracy: 0.20711391265195858 Regularization: 0.05
Best accuracy: 0.24245835209365152 Regularization: 0.1
Best accuracy: 0.285006753714543 Regularization: 0.5
Best accuracy: 0.2915353444394417 Regularization: 1


LogisticRegression(C=1, penalty='l1', solver='liblinear')

#### Predict only 16 scene categories with all part/object attributes

In [101]:
hyperparam_search_l1(
    train_features=attributes['train'],
    train_labels=category_predictions['train'],
    val_features=attributes['val'],
    val_labels=category_predictions['val'])

Best accuracy: 0.3275551553354345 Regularization: 0.001
Best accuracy: 0.39756866276452046 Regularization: 0.005
Best accuracy: 0.43876632147681227 Regularization: 0.01
Best accuracy: 0.5538045925258892 Regularization: 0.05
Best accuracy: 0.5839711841512832 Regularization: 0.1
Best accuracy: 0.6226924808644755 Regularization: 0.5
Best accuracy: 0.6292210715893741 Regularization: 1


LogisticRegression(C=1, penalty='l1', solver='liblinear')

#### Predict 16 scene categories with top attributes (appear in 150 or more training examples)

In [102]:
hyperparam_search_l1(
    train_features=freq_attributes['train'],
    train_labels=category_predictions['train'],
    val_features=freq_attributes['val'],
    val_labels=category_predictions['val'])

Best accuracy: 0.3275551553354345 Regularization: 0.001
Best accuracy: 0.39756866276452046 Regularization: 0.005
Best accuracy: 0.43876632147681227 Regularization: 0.01
Best accuracy: 0.5319675821701936 Regularization: 0.05
Best accuracy: 0.5522287257991896 Regularization: 0.1
Best accuracy: 0.5621341737955876 Regularization: 0.5


LogisticRegression(C=0.5, penalty='l1', solver='liblinear')

### Obtain predictions of linear classifier on validation set (16 way classifier with only frequent attributes)

In [9]:
# Define variables
c = 0.5
train_X = freq_attributes['train']
val_X = freq_attributes['val']

train_y = category_predictions['train']
val_y = category_predictions['val']

baseline_save_dir = os.path.join(save_dir, 'baseline_explainer')
ensure_dir(baseline_save_dir)
param_string = '16_class_freq_attr'

incongruent_save_path = os.path.join(baseline_save_dir, 'incongruent_paths_{}.txt'.format(param_string))
congruent_save_path = os.path.join(baseline_save_dir, 'congruent_paths_{}.txt'.format(param_string))

# Create logistic regression classifier and predict for validation set
logreg = LogisticRegression(solver='liblinear', C=c, penalty='l1')
logreg.fit(train_X, train_y)

logreg_category_predictions = logreg.predict(val_X)

n_samples = len(val_y)
assert len(logreg_category_predictions) == n_samples
assert len(val_paths) == n_samples

print("Accuracy: {}".format(1 - np.count_nonzero(logreg_category_predictions != val_y) / n_samples))



Accuracy: 0.5621341737955876


In [ ]:
incongruent_paths = []
congruent_paths = []
for logreg_pred, model_pred, image_path in tqdm(zip(logreg_category_predictions, val_y, val_paths)):
    if logreg_pred != model_pred:
        incongruent_paths.append(image_path)
    else:
        congruent_paths.append(image_path)

for save_path, paths_list in zip([incongruent_save_path, congruent_save_path], [incongruent_paths, congruent_paths]):
    if os.path.exists(save_path):
        print("{} already exists".format(save_path))
    else:
        write_lists(save_path, paths_list)
        print("Saved {} paths to {}".format(len(paths_list), save_path))

#### Obtain the pre-normalized output and normalized probabilities from the logistic regressor on validation

In [11]:
outputs = logreg.decision_function(val_X)
output_probabilities = logreg.predict_proba(val_X)
print(outputs[0], output_probabilities[0], np.sum(output_probabilities[0]))

[ -6.21541755  -9.57385951 -11.01464339  -8.19257013  -8.35659567
  -2.3154842   -5.336674    -7.39402475  -2.75187164  -5.73843319
  -3.09532866   0.10664419  -1.61541948  -4.4823798   -1.89333494
  -2.06268626] [1.73180361e-03 6.03646460e-05 1.42916118e-05 2.40203346e-04
 2.03873998e-04 7.80187178e-02 4.15828632e-03 5.33624900e-04
 5.20836190e-02 2.78689906e-03 3.75992050e-02 4.57296199e-01
 1.44002514e-01 9.70804106e-03 1.13634363e-01 9.79279926e-02] 1.0


In [17]:
def aggregate_category_logits(outputs, category_scene_dict, aggregation_mode='sum'):
    '''
    Given classifier outputs and scene mapper, aggregate logit outputs based on aggregation mode
    '''
    if aggregation_mode == 'sum':
        agg_fn = np.sum
    elif aggregation_mode == 'mean':
        agg_fn = np.mean
    else:
        raise ValueError("Unsupported aggregation mode '{}'".format(aggregation_mode))
    
    aggregated_outputs = []
    sorted_keys = sorted(list(category_scene_dict.keys()))
    for category in sorted_keys:
        # print(category)
        scenes = category_scene_dict[category]
        category_aggregated = agg_fn(outputs[:, scenes], axis=1)
        # print(category_aggregated.shape)
        aggregated_outputs.append(category_aggregated)
        
    aggregated_outputs = np.stack(aggregated_outputs, axis=1)
    return aggregated_outputs


category_scene_dict = get_category_class_dict()

mean_category_logits = aggregate_category_logits(
    outputs=val_logits,
    category_scene_dict=category_scene_dict,
    aggregation_mode='mean')

mean_argmax = np.argmax(mean_category_logits, axis=1)
print(mean_argmax.shape)
sum_category_logits = aggregate_category_logits(
    outputs=val_logits,
    category_scene_dict=category_scene_dict,
    aggregation_mode='sum')
sum_argmax = np.argmax(sum_category_logits, axis=1)

print(np.count_nonzero(mean_argmax - sum_argmax))
print(np.count_nonzero(mean_argmax - val_predictions))
print(np.count_nonzero(sum_argmax - val_predictions))

print(mean_argmax[:5], val_predictions[:5])

(4442,)
1103
4437
4434
[ 8 15 15  2  2] [209 112 125  52  45]
